In [1]:
from pathlib import Path, PurePath
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from analysis import read_behaviors, read_moea_fitness, unpickle_moea
from utility.util_functions import calc_str_disparity, euclidean_distance
from utility.visualize import plot_path, plot_moea_fitness, plot_surrogate_model, plot_str_disparity, plot_moea_fitness_2d
import pickle
import numpy as np
from deap import base, creator
from evolution.simulation import Disparity
import scipy as sp
from scipy import stats
import math
init_notebook_mode(connected=True)


ModuleNotFoundError: DBus works only on linux!


Using TensorFlow backend.


ModuleNotFoundError: DBus works only on linux!


In [2]:
# get the path for all behavioral features
files = sorted(Path('data/results/').glob('transferability_*/behavioral_features.dat'))
fitness_f = sorted(Path('data/results/').glob('transferability_*/fitness.dat'))

In [3]:
columns = [
        'avg_left', 'avg_right',
        's1', 's2', 's3', 's4', 's5', 's6', 's7',
        'area0_percentage',
        'area1_percentage',
        'area2_percentage',
    ]

In [27]:
fitness_f[6]

PosixPath('data/results/transferability_simulation_15/fitness.dat')

In [28]:
behaviors = read_behaviors([files[6]])[0]

In [ ]:
behaviors.loc[behaviors['genome_id'] == 494].head()

In [ ]:
behaviors.head()

In [ ]:
b_grouped = behaviors.groupby(['genome_id']).filter(lambda x: len(x) > 1).groupby(['genome_id', 'simulation']).first()

In [ ]:
b_grouped.head()

In [ ]:
euclidean_distance(b_grouped[columns].iloc[1], b_grouped[columns].iloc[0])

In [ ]:
for i in range(1, len(b_grouped), 2):
    print('genome_id:', 
          b_grouped[columns].iloc[i].name[0],
          'actual diversity',
          euclidean_distance(
              b_grouped[columns].iloc[i],
              b_grouped[columns].iloc[i-1]
          )
    )

In [29]:
fitness_data = read_moea_fitness([fitness_f[6]])[0]

In [ ]:
fitness_data.sort_values(by=['fitness'])

In [30]:
fitness_data.head()

,gen,genome_id,fitness,str_disparity,diversity
0,0,1,0.083543,0.119517,0.850386
1,0,2,0.197851,0.119517,1.472908
2,0,3,0.109647,0.119517,0.820689
3,0,4,0.355871,0.119517,1.755296
4,0,5,0.035187,0.119517,0.878536


In [31]:
hof = unpickle_moea('data/results/transferability_simulation_15/hof')

In [32]:
len(hof)

12

In [38]:
plot_moea_fitness_2d(fitness_data, hof)

In [39]:
hof_ids = []
for ind in hof:
    hof_ids.append(ind.key)
    print(ind.key, ind.fitness.values)

367 (8.179329914947463, 1.5393718591076808, 0.29039993833122046)
358 (8.1341098262261, 1.5393718591076808, 0.32162270476427535)
532 (7.972001420778755, 1.474114958028711, 0.2686715795477769)
72 (7.925807476793328, 0.31058753450850884, 0.4509520485424759)
324 (7.729498866710899, 1.5393718591076808, 0.4622053138150647)
311 (7.6351439208980425, 0.5941746403598056, 0.5715480416713524)
63 (5.790913648047852, 0.31058753450850884, 1.0845821479478892)
45 (5.344447774265978, 0.29623176569279985, 0.6380325799257398)
26 (1.15842073060853, 0.11951712563775475, 0.5239408618426444)
16 (0.5547505879621605, 0.11951712563775475, 1.737585530121403)
25 (0.4379590499598192, 0.11951712563775475, 1.7735880839622107)
40 (0.2989427541577952, 0.11951712563775475, 1.77920024534944)


In [40]:
genomes_file = sorted(Path('data/results/transferability_simulation_15/deap_inds').glob('*_genome.pkl'))
transfered_file = sorted(Path('data/results/transferability_simulation_15/deap_inds').glob('*_transformed_genome_.pkl'))
genomes = [unpickle_moea(genome) for genome in genomes_file]
transferred = [unpickle_moea(genome) for genome in transfered_file]

In [41]:
transfered_file

[PosixPath('data/results/transferability_simulation_15/deap_inds/109_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_15/deap_inds/112_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_15/deap_inds/119_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_15/deap_inds/123_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_15/deap_inds/138_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_15/deap_inds/141_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_15/deap_inds/142_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_15/deap_inds/148_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_15/deap_inds/155_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_15/deap_inds/158_transformed_genome_.pkl'),
 PosixPath('data/results/transferability

In [43]:
len(transfered_file)

78

In [87]:
g = sorted(genomes, key=lambda x: x.key)

In [88]:
t = sorted(transferred, key=lambda x: x.key)

In [89]:
sim_genome = list(filter(lambda x: x.key == 530, g))[0]

In [90]:
print(sim_genome.key, sim_genome.fitness.values, sim_genome.sim_time, sim_genome.diversity)

530 (1.3232420685644013, 1.474114958028711, 0.590481990508962) 11.200000000000024 0.590481990508962


In [78]:
trans_genome = list(filter(lambda x: x.key == 375, t))[0]

In [79]:
print(trans_genome.key, trans_genome.fitness.values)

375 (5.920783593410858, 1.5393718591076808, 0.5018630528981484)


In [58]:
sim_genome.sim_time

60.049999999998725

In [ ]:
sim_genomes = sorted(list(filter(lambda x: x.key in hof_ids[:-10], g)), key=lambda x: x.fitness.values[0])
trans_genomes = sorted(list(filter(lambda x: x.key in hof_ids[:-10] t)), key=lambda x: x.fitness.values[0])

In [80]:
plot_path([sim_genome, trans_genome], 'The discrepancy of traveled trajectories in simulation & reality.')
# plot_path(sim_genomes)

In [ ]:
sim_genome.position

In [ ]:
trans_genome.position

In [ ]:
plot_path(trans_genomes)

In [60]:
calc_str_disparity(trans_genome.position, sim_genome.position)

22.447773190834052

In [61]:
euclidean_distance(trans_genome.position, sim_genome.position)

0.2771459559699877

In [91]:
plot_surrogate_model(fitness_data)

In [71]:
# merge  
union_t = [(x, y) for x in g for y in t if x.key == y.key]
for s, t in union_t:
    np.array_equal(s, t)
    assert s.key == t.key

In [63]:
# merge HOF
union_hof = [(x, y) for x in hof for y in t if x.key == y.key]
for s, t in union_hof:
    np.array_equal(s, t)
    assert s.key == t.key

In [64]:
len(union_hof)

6

In [ ]:
for s, t in union_hof:
    print(s.key, end=" ")

In [72]:
str_disparities = [(s.gen,
                    s.key,
                    s.fitness.values[0],
                    s.fitness.values[1],
                    calc_str_disparity(t.position, s.position)
                   ) for s, t in union_t
                  ]

In [ ]:
str_euclidean = [(s.gen,
                    s.key,
                    s.fitness.values[0],
                    s.fitness.values[1],
                    calc_str_disparity(t.position, s.position)
                   ) for s, t in union_t
                ]

In [67]:
str_disparities_hof = [(s.gen,
                    s.key,
                    s.fitness.values[0],
                    s.fitness.values[1],
                    calc_str_disparity(t.position, s.position)
                   ) for s, t in union_hof
                  ]

In [ ]:
sorted(str_disparities, key=lambda x: x[0])

In [ ]:
np.array([str_disparity[1] for str_disparity in str_disparities])
np.array([real_disparity[2] for real_disparity in str_disparities])

filter_disparities = list(filter(lambda x: x[2] < 5, str_disparities))
filter_disparities[:4]

In [73]:
plot_str_disparity(str_disparities)

In [81]:
actual = []
approximated = []
for s, t in union_t:
    approximated.append(s.fitness.values[1])
    actual.append(calc_str_disparity(t.position, s.position))

In [82]:
stats.pearsonr(actual, approximated)

(0.016407392036827017, 0.887373147485877)

In [ ]:
str_euclidean

In [ ]:
distance = 0
for i in range(0, len(trans_genome.position)-1):
    distance += math.sqrt(((sim_genome.position[i][0] - 
                            sim_genome.position[i][1])**2) + 
                          ((sim_genome.position[i+1][1] - 
                            sim_genome.position[i+1][1])**2))

In [84]:
def traveled_distance(genome):
    distance = 0
    for i in range(0, len(genome.position)-1):
        distance += math.sqrt(((genome.position[i][0] - 
                                genome.position[i][1])**2) + 
                              ((genome.position[i+1][1] - 
                                genome.position[i+1][1])**2))
    return distance

In [85]:
traveled_distance(trans_genome)

0.4346722693029195

In [86]:
traveled_distance(sim_genome)

2.814394548535347